In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [3]:
# 시각화 관련
import plotly_express as px
import plotly.figure_factory as ff
import cufflinks as cf
cf.go_offline(connected=True)
## cf.getThemes()
cf.set_config_file(theme='polar')
import plotly.graph_objects as go

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
## plt.style.use("fivethirtyeight")
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["figure.figsize"] = (10,6)

In [5]:
path = 'C:/Users/sdjhs/git'
trading_df = pd.read_csv(f"{path}/data/DL/매매내역_서울.csv")

trading_df['평형대코드'] = '5'
trading_df['평형대코드'][trading_df['전용면적'] < 136] = '4'
trading_df['평형대코드'][trading_df['전용면적'] < 85] = '3'
trading_df['평형대코드'][trading_df['전용면적'] < 60] = '2'
trading_df['평형대코드'][trading_df['전용면적'] < 48] = '1'

In [6]:
## 마스터 테이블
danji_mst = trading_df.sort_values(by=['년', '월', '일', ], ascending=False)[['일련번호', '전용면적', '평형대코드', '아파트', '건축년도', '등기일자',
            '도로명시군구코드', '도로명일련번호코드', '도로명지상지하코드', '도로명코드', '도로명', '도로명건물본번호코드', '도로명건물부번호코드',
            '법정동시군구코드', '법정동읍면동코드', '법정동지번코드','법정동본번코드', '법정동부번코드', '법정동', 
            '지번', '해제여부', '해제사유발생일']].drop_duplicates(subset=['일련번호', '전용면적', '아파트']).copy()

In [7]:
## 거래내역 원천 테이블
trading_trs = trading_df[['년', '월', '일', '일련번호', '평형대코드','전용면적', '동', '층', '거래금액']][trading_df['거래유형'] !='직거래'].copy()
trading_trs['거래금액'] = trading_trs['거래금액'].apply(lambda x: int(x.replace(',', '')))

In [8]:
## 필요컬럼 전처리
trading_trs_v2 = pd.pivot_table(trading_trs, index=['년', '월', '일련번호', '평형대코드', '전용면적'], values='거래금액', aggfunc=np.mean).reset_index()
trading_trs_v2['기준년월'] = pd.to_datetime(trading_trs_v2['년'].astype('str') + '-' + trading_trs_v2['월'].astype('str'))
trading_trs_v2['평당매매거래금액'] = (trading_trs_v2['거래금액']/trading_trs_v2['전용면적'])*3.3045
trading_trs_v2 = trading_trs_v2.merge(danji_mst[['일련번호', '법정동시군구코드', '법정동읍면동코드']].drop_duplicates().copy(), on='일련번호', how='left')

In [9]:
## 아우터 조인
_p1 = trading_trs_v2[['기준년월']].drop_duplicates()
_p1['dummy'] = 1
_p2 = trading_trs_v2[['일련번호', '평형대코드', '법정동시군구코드', '법정동읍면동코드']].drop_duplicates()
_p2['dummy'] = 1
trading_trs_v3 = pd.merge(_p1, _p2, on='dummy').drop('dummy', axis=1)

## 단위별 평당매매거래금액 집계
trading_trs_danji = pd.pivot_table(trading_trs_v2, index=['기준년월', '일련번호', '평형대코드'], values='평당매매거래금액', aggfunc=np.mean).reset_index()
trading_trs_bjd = pd.pivot_table(trading_trs_v2, index=['기준년월', '법정동시군구코드', '법정동읍면동코드', '평형대코드'],
                                 values='평당매매거래금액', aggfunc=np.mean).rename(columns={'평당매매거래금액':'읍면동평균매매거래금액'}).reset_index()
trading_trs_sgg = pd.pivot_table(trading_trs_v2, index=['기준년월', '법정동시군구코드', '평형대코드'],
                                 values='평당매매거래금액', aggfunc=np.mean).rename(columns={'평당매매거래금액':'시군구평균매매거래금액'}).reset_index()
trading_trs_v3 = trading_trs_v3.merge(trading_trs_danji, how='left').merge(trading_trs_bjd, how='left').merge(trading_trs_sgg, how='left')

## 단위별 대표값 교환비
ratio = trading_trs_v3.copy()
ratio['시군구교환비'] = ratio['평당매매거래금액']/ratio['시군구평균매매거래금액']
ratio['읍면동교환비'] = ratio['평당매매거래금액']/ratio['읍면동평균매매거래금액']
ratio = pd.pivot_table(ratio, index=['일련번호', '평형대코드', '법정동시군구코드', '법정동읍면동코드'],
                       values=['시군구교환비', '읍면동교환비'],
                       aggfunc=np.mean).reset_index()

trading_trs_v3 = pd.merge(trading_trs_v3, ratio, on=['일련번호', '평형대코드', '법정동시군구코드', '법정동읍면동코드'], how='left')
trading_trs_v3['보간읍면동매매거래금액'] = trading_trs_v3['읍면동평균매매거래금액'] * trading_trs_v3['읍면동교환비']
trading_trs_v3['보간시군구매매거래금액'] = trading_trs_v3['시군구평균매매거래금액'] * trading_trs_v3['시군구교환비']

trading_trs_v3['평당매매거래금액'][trading_trs_v3['평당매매거래금액'].isna()] = trading_trs_v3['읍면동평균매매거래금액'][trading_trs_v3['평당매매거래금액'].isna()]
trading_trs_v3['평당매매거래금액'][trading_trs_v3['평당매매거래금액'].isna()] = trading_trs_v3['시군구평균매매거래금액'][trading_trs_v3['평당매매거래금액'].isna()]

trading_trs_v3['단지평형대코드'] = trading_trs_v3['일련번호'] + '/' + trading_trs_v3['평형대코드']
trading_trs_v3 = pd.pivot_table(trading_trs_v3, index=['기준년월'], columns=['단지평형대코드'], values='평당매매거래금액', aggfunc=np.mean)
trading_trs_v3 = trading_trs_v3.interpolate(method='linear', axis=1, limit_direction='both')
trading_trs_v3 = trading_trs_v3.reset_index().melt(id_vars=['기준년월'], value_name='평당매매거래금액')

trading_trs_v3['일련번호'] = trading_trs_v3['단지평형대코드'].apply(lambda x: x.split('/')[0])
trading_trs_v3['평형대코드'] = trading_trs_v3['단지평형대코드'].apply(lambda x: x.split('/')[1])
trading_trs_v3 = trading_trs_v3.drop('단지평형대코드', axis=1)

In [10]:
danji_mst[['일련번호', '아파트', '전용면적', '평형대코드']].merge(trading_trs_v3, how='outer')

,일련번호,아파트,전용면적,평형대코드,기준년월,평당매매거래금액
0,11110-10,옥인시민,39.67,1,2006-01-01,1238.641058
1,11110-10,옥인시민,39.67,1,2006-02-01,1365.921604
2,11110-10,옥인시민,39.67,1,2006-03-01,1606.354167
3,11110-10,옥인시민,39.67,1,2006-04-01,836.051758
4,11110-10,옥인시민,39.67,1,2006-05-01,1403.783047
...,...,...,...,...,...,...
8035326,11740-99,중앙하이츠,104.36,4,2023-09-01,4678.176841
8035327,11740-99,중앙하이츠,104.36,4,2023-10-01,4421.327268
8035328,11740-99,중앙하이츠,104.36,4,2023-11-01,4527.092846
8035329,11740-99,중앙하이츠,104.36,4,2023-12-01,2311.503450


In [ ]:
## 전용면적 * 평당매매거래금액 => 라인, 분석용
## 원천거래금액 => 포인트용

In [402]:
trading_trs

,년,월,일,일련번호,평형대코드,전용면적,동,층,거래금액
0,2006,2,6,11110-205,1,40.07,NaN,7,22555
1,2006,2,2,11110-16,4,90.13,NaN,8,24813
2,2006,2,10,11110-16,4,90.13,NaN,8,24813
3,2006,2,17,11110-17,4,108.89,NaN,9,30000
4,2006,2,5,11110-121,3,84.42,NaN,6,22000
...,...,...,...,...,...,...,...,...,...
1275275,2023,12,28,11740-2628,3,84.99,NaN,7,133500
1275276,2023,12,30,11740-170,3,84.97,NaN,4,75950
1275277,2023,12,20,11740-2852,3,84.53,NaN,9,87000
1275278,2023,12,23,11740-4743,2,49.82,402,6,65000


,기준년월,단지평형대코드,평당매매거래금액
0,2006-01-01,11110-10-1,1238.641058
1,2006-02-01,11110-10-1,1365.921604
2,2006-03-01,11110-10-1,1606.354167
3,2006-04-01,11110-10-1,836.051758
4,2006-05-01,11110-10-1,1403.783047
...,...,...,...
3441398,2023-09-01,11740-99-4,4678.176841
3441399,2023-10-01,11740-99-4,4421.327268
3441400,2023-11-01,11740-99-4,4527.092846
3441401,2023-12-01,11740-99-4,2311.503450


In [332]:
trading_trs_v3[(trading_trs_v3['일련번호'] == '11440-223')&(trading_trs_v3['평형대코드'] == '5')].dropna()

,기준년월,일련번호,평형대코드,법정동시군구코드,법정동읍면동코드,평당매매거래금액,읍면동평균매매거래금액,시군구평균매매거래금액,시군구교환비,읍면동교환비
1535897,2014-01-01,11440-223,5,11440,10200,38.812544,1151.780329,2058.964201,0.018851,0.033698


In [356]:
trading_df[trading_df['일련번호'] == '11440-223'][['년', '거래금액']]

,년,거래금액
584901,2014,"2,000"
612176,2022,"35,000"


In [337]:
trading_df

,거래금액,거래유형,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,...,월,일,일련번호,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"22,555",NaN,2005,2006,율곡로2길,7.0,0.0,11110.0,1.0,0.0,...,2,6,11110-205,40.07,NaN,85,11110,7,NaN,NaN
1,"24,813",NaN,1998,2006,삼일대로32길,36.0,0.0,11110.0,2.0,0.0,...,2,2,11110-16,90.13,NaN,30-6,11110,8,NaN,NaN
2,"24,813",NaN,1998,2006,삼일대로32길,36.0,0.0,11110.0,2.0,0.0,...,2,10,11110-16,90.13,NaN,30-6,11110,8,NaN,NaN
3,"30,000",NaN,1968,2006,삼일대로,428.0,0.0,11110.0,5.0,0.0,...,2,17,11110-17,108.89,NaN,288,11110,9,NaN,NaN
4,"22,000",NaN,2004,2006,율곡로19길,41.0,6.0,11110.0,1.0,0.0,...,2,5,11110-121,84.42,NaN,9-1,11110,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275275,"133,500",중개거래,2007,2023,올림픽로89길,57.0,0.0,11740.0,1.0,0.0,...,12,28,11740-2628,84.99,서울 강동구,568,11740,7,NaN,NaN
1275276,"75,950",중개거래,2000,2023,구천면로,278.0,0.0,11740.0,4.0,0.0,...,12,30,11740-170,84.97,서울 강동구,217-132,11740,4,NaN,NaN
1275277,"87,000",중개거래,2009,2023,아리수로93길,40.0,0.0,11740.0,1.0,0.0,...,12,20,11740-2852,84.53,서울 강동구,674,11740,9,NaN,NaN
1275278,"65,000",중개거래,2020,2023,아리수로93가길,110.0,0.0,11740.0,0.0,NaN,...,12,23,11740-4743,49.82,서울 강동구,114,11740,6,NaN,NaN


In [176]:
ratio['읍면동평균매매거래금액']/ratio['평당매매거래금액']

In [197]:
pd.merge(trading_trs_v3_sgg, trading_trs_v3_bjd, on=['기준년월', '법정동시군구코드','평형대코드'])

,기준년월,법정동시군구코드,평형대코드,시군구평균매매거래금액,법정동읍면동코드,읍면동평균매매거래금액
0,2006-01-01,11110,1,1111.099468,17100,1493.724236
1,2006-01-01,11110,1,1111.099468,17500,728.474701
2,2006-01-01,11110,2,943.098970,13300,909.691055
3,2006-01-01,11110,2,943.098970,17400,989.235603
4,2006-01-01,11110,2,943.098970,18300,966.712638
...,...,...,...,...,...,...
148830,2024-01-01,11740,4,3991.318470,10200,5267.476733
148831,2024-01-01,11740,4,3991.318470,10600,3403.829947
148832,2024-01-01,11740,4,3991.318470,10700,4501.274501
148833,2024-01-01,11740,4,3991.318470,10800,3573.759586


In [181]:
trading_trs_v3_sgg

,기준년월,법정동시군구코드,평형대코드,평당매매거래금액
0,2006-01-01,11110,1,1111.099468
1,2006-01-01,11110,2,943.098970
2,2006-01-01,11110,3,1104.981718
3,2006-01-01,11110,4,1157.247371
4,2006-01-01,11140,1,611.372946
...,...,...,...,...
25908,2024-01-01,11710,5,5765.079753
25909,2024-01-01,11740,1,2817.169894
25910,2024-01-01,11740,2,5258.827560
25911,2024-01-01,11740,3,4027.182915


In [182]:
trading_trs_v3_bjd

,기준년월,법정동읍면동코드,평형대코드,평당매매거래금액
0,2006-01-01,10100,1,1126.435756
1,2006-01-01,10100,2,1280.691139
2,2006-01-01,10100,3,1443.550701
3,2006-01-01,10100,4,1743.978447
4,2006-01-01,10100,5,2078.523279
...,...,...,...,...
39167,2024-01-01,17500,3,3818.675230
39168,2024-01-01,17700,3,7790.334045
39169,2024-01-01,17900,2,8612.556742
39170,2024-01-01,17900,3,7595.575694


In [179]:
trading_trs_v3

,기준년월,일련번호,평형대코드,평당매매거래금액,법정동시군구코드,법정동읍면동코드
0,2006-01-01,11110-13,4,1043.712527,11110,11700
1,2006-01-01,11110-131,1,728.474701,11110,17500
2,2006-01-01,11110-16,2,909.691055,11110,13300
3,2006-01-01,11110-16,3,909.716506,11110,13300
4,2006-01-01,11110-26,1,1493.724236,11110,17100
...,...,...,...,...,...,...
600579,2024-01-01,11740-7,3,5506.987058,11740,10100
600580,2024-01-01,11740-71,2,3297.629938,11740,10600
600581,2024-01-01,11740-79,3,3956.312233,11740,10600
600582,2024-01-01,11740-90,2,5037.853912,11740,10700


,일련번호,법정동시군구코드,법정동읍면동코드
93179,11200-19,11200,10700
269386,11290-82,11290,13300
354635,11320-21,11320,10500
354647,11320-42,11320,10600
354679,11320-98,11320,10700
...,...,...,...
982462,11650-29,11650,10800
843668,11560-26,11560,11000
940290,11620-3965,11620,10100
817427,11545-38,11545,10300


In [147]:
trading_trs_v3

일련번호       11110-10             11110-100    11110-101    11110-102  \
평형대코드             1           2         2            5            1   
기준년월                                                                  
2006-01-01      NaN         NaN       NaN          NaN          NaN   
2006-02-01      NaN         NaN       NaN          NaN          NaN   
2006-03-01      NaN         NaN       NaN  1033.772254          NaN   
2006-04-01      NaN         NaN       NaN          NaN          NaN   
2006-05-01      NaN  1160.66642       NaN          NaN  1048.597770   
...             ...         ...       ...          ...          ...   
2023-09-01      NaN         NaN       NaN          NaN          NaN   
2023-10-01      NaN         NaN       NaN          NaN  2729.326497   
2023-11-01      NaN         NaN       NaN          NaN  2798.622427   
2023-12-01      NaN         NaN       NaN          NaN          NaN   
2024-01-01      NaN         NaN       NaN          NaN          NaN   

일련번호                    11110-105            11110-107  ... 11740-96      \
평형대코드                 2         1   2   3            4  ...        2   3   
기준년월                                                    ...                
2006-01-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2006-02-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2006-03-01          NaN       NaN NaN NaN  2197.973403  ...      NaN NaN   
2006-04-01  1021.168109       NaN NaN NaN  1130.478462  ...      NaN NaN   
2006-05-01          NaN       NaN NaN NaN  2244.613892  ...      NaN NaN   
...                 ...       ...  ..  ..          ...  ...      ...  ..   
2023-09-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2023-10-01  2961.387515       NaN NaN NaN          NaN  ...      NaN NaN   
2023-11-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2023-12-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2024-01-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   

일련번호       11740-969              11740-97     11740-98              \
평형대코드              2            3        3   5        2           3   
기준년월                                                                  
2006-01-01       NaN          NaN      NaN NaN      NaN  990.320067   
2006-02-01       NaN          NaN      NaN NaN      NaN         NaN   
2006-03-01       NaN   945.895369      NaN NaN      NaN         NaN   
2006-04-01       NaN          NaN      NaN NaN      NaN         NaN   
2006-05-01       NaN          NaN      NaN NaN      NaN         NaN   
...              ...          ...      ...  ..      ...         ...   
2023-09-01       NaN          NaN      NaN NaN      NaN         NaN   
2023-10-01       NaN          NaN      NaN NaN      NaN         NaN   
2023-11-01       NaN  3283.059611      NaN NaN      NaN         NaN   
2023-12-01       NaN          NaN      NaN NaN      NaN         NaN   
2024-01-01       NaN          NaN      NaN NaN      NaN         NaN   

일련번호           11740-99              
평형대코드                 2           4  
기준년월                                 
2006-01-01   899.410282         NaN  
2006-02-01   893.858367         NaN  
2006-03-01   975.749118         NaN  
2006-04-01   888.306452         NaN  
2006-05-01          NaN         NaN  
...                 ...         ...  
2023-09-01          NaN         NaN  
2023-10-01          NaN         NaN  
2023-11-01  3109.072581         NaN  
2023-12-01          NaN  2311.50345  
2024-01-01          NaN         NaN  

[217 rows x 15859 columns]

일련번호       11110-10             11110-100    11110-101    11110-102  \
평형대코드             1           2         2            5            1   
기준년월                                                                  
2006-01-01      NaN         NaN       NaN          NaN          NaN   
2006-02-01      NaN         NaN       NaN          NaN          NaN   
2006-03-01      NaN         NaN       NaN  1033.772254          NaN   
2006-04-01      NaN         NaN       NaN          NaN          NaN   
2006-05-01      NaN  1160.66642       NaN          NaN  1048.597770   
...             ...         ...       ...          ...          ...   
2023-09-01      NaN         NaN       NaN          NaN          NaN   
2023-10-01      NaN         NaN       NaN          NaN  2729.326497   
2023-11-01      NaN         NaN       NaN          NaN  2798.622427   
2023-12-01      NaN         NaN       NaN          NaN          NaN   
2024-01-01      NaN         NaN       NaN          NaN          NaN   

일련번호                    11110-105            11110-107  ... 11740-96      \
평형대코드                 2         1   2   3            4  ...        2   3   
기준년월                                                    ...                
2006-01-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2006-02-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2006-03-01          NaN       NaN NaN NaN  2197.973403  ...      NaN NaN   
2006-04-01  1021.168109       NaN NaN NaN  1130.478462  ...      NaN NaN   
2006-05-01          NaN       NaN NaN NaN  2244.613892  ...      NaN NaN   
...                 ...       ...  ..  ..          ...  ...      ...  ..   
2023-09-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2023-10-01  2961.387515       NaN NaN NaN          NaN  ...      NaN NaN   
2023-11-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2023-12-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   
2024-01-01          NaN       NaN NaN NaN          NaN  ...      NaN NaN   

일련번호       11740-969              11740-97     11740-98              \
평형대코드              2            3        3   5        2           3   
기준년월                                                                  
2006-01-01       NaN          NaN      NaN NaN      NaN  990.320067   
2006-02-01       NaN          NaN      NaN NaN      NaN         NaN   
2006-03-01       NaN   945.895369      NaN NaN      NaN         NaN   
2006-04-01       NaN          NaN      NaN NaN      NaN         NaN   
2006-05-01       NaN          NaN      NaN NaN      NaN         NaN   
...              ...          ...      ...  ..      ...         ...   
2023-09-01       NaN          NaN      NaN NaN      NaN         NaN   
2023-10-01       NaN          NaN      NaN NaN      NaN         NaN   
2023-11-01       NaN  3283.059611      NaN NaN      NaN         NaN   
2023-12-01       NaN          NaN      NaN NaN      NaN         NaN   
2024-01-01       NaN          NaN      NaN NaN      NaN         NaN   

일련번호           11740-99              
평형대코드                 2           4  
기준년월                                 
2006-01-01   899.410282         NaN  
2006-02-01   893.858367         NaN  
2006-03-01   975.749118         NaN  
2006-04-01   888.306452         NaN  
2006-05-01          NaN         NaN  
...                 ...         ...  
2023-09-01          NaN         NaN  
2023-10-01          NaN         NaN  
2023-11-01  3109.072581         NaN  
2023-12-01          NaN  2311.50345  
2024-01-01          NaN         NaN  

[217 rows x 15859 columns]

In [43]:
tt = pd.merge(temp, danji_mst[['단지명']], left_on='아파트', right_on='단지명', how='left')

In [46]:
tt[tt['단지명'].isna()]

,일련번호,아파트,법정동,도로명,단지명
1,11110-16,운현신화타워,익선동,삼일대로32길,NaN
2,11110-17,낙원,낙원동,삼일대로,NaN
3,11110-121,송림아마레스아파트,이화동,율곡로19길,NaN
4,11110-26,명륜동주상복합아남아파트,명륜2가,혜화로3길,NaN
5,11110-25,아남1,명륜2가,창경궁로,NaN
...,...,...,...,...,...
9329,11740-4804,강동리버스트8단지,강일동,아리수로93나길,NaN
9330,11740-4648,둔촌현대수린나아파트,둔촌동,동남로49길,NaN
9332,11740-4951,고덕아르테스미소지움,상일동,동남로,NaN
9333,11740-5039,거산유팰리스2차,길동,천호대로177길,NaN
